# Model evaluation
I want to find out which model is the best for my data. For this I will check different models and compare them with each other.

## Import of required libraries
Those libraries are used in the notebook to perform the data preparation. Maybee you need to install them first with `pip install <library>`.  

In [6]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib
import matplotlib.pyplot as plt
import nltk 
import seaborn as sns
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
%matplotlib inline


# Load Huggingface transformers
from transformers import TFBertModel,  BertConfig, BertTokenizerFast, TFAutoModel

# Then what you need from tensorflow.keras
from tensorflow.keras.layers import Input, Dropout, Dense, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical

pd.set_option('display.max_colwidth', 100)

import re,string,unicodedata
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier,ExtraTreesClassifier
from collections import Counter
import string

import warnings
warnings.filterwarnings('ignore')
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet
# sklearn
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from mlxtend.plotting import plot_confusion_matrix
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score,f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import warnings
from sklearn.pipeline import Pipeline

## Import Dataset
As first step I will import the cleaned dataset from the previous notebook "01_data_prep".

In [8]:
new_data = pd.read_csv('../pythoncourse/04_Data Cleansing/20_model_evaluation/20_cleaned_data.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../pythoncourse/04_Data Cleansing/20_model_evaluation/20_cleaned_data.csv'

In [ ]:
# Name of the BERT model to use
model_name = 'bert-base-uncased'

# Max length of tokens
max_length = 128

# Load transformers config and set output_hidden_states to False
config = BertConfig.from_pretrained(model_name)
#config.output_hidden_states = False

# Load BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = model_name, config = config)
bert = TFAutoModel.from_pretrained(model_name)

In [ ]:
input_ids = Input(shape=(max_length,), name='input_ids', dtype='int32')
attention_mask = Input(shape=(max_length,), name='attention_mask', dtype='int32') 
inputs = {'input_ids': input_ids, 'attention_mask': attention_mask}
x = bert.bert(inputs)

In [ ]:
all_data_df

In [ ]:
df_la = pd.get_dummies(all_data_df, columns = ['cyberbullying_type'])
df_la

In [ ]:
train_sentences = all_data_df["cleaned"].values
list_classes = ['cyberbullying_type_age','cyberbullying_type_ethnicity','cyberbullying_type_gender','cyberbullying_type_not_cyberbullying','cyberbullying_type_other_cyberbullying','cyberbullying_type_religion']
train_y = df_la[list_classes].values

In [ ]:
#x2 =Dense(512, activation='relu')(x[1])
x2 = GlobalAveragePooling1D()(x[0])
#x3 = Dropout(0.5)(x2)
y =Dense(len(list_classes), activation='sigmoid', name='outputs')(x2)

model = Model(inputs=inputs, outputs=y)
#model.layers[2].trainable = False

# Take a look at the model
model.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam

# Verwende den neuen Adam-Optimizer mit learning_rate
optimizer = Adam(learning_rate=1e-5)

model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])


In [ ]:
# Tokenize the input 
x = tokenizer(
    text=list(train_sentences),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [ ]:
history = model.fit(
    x={'input_ids': x['input_ids'], 'attention_mask': x['attention_mask']},
    #x={'input_ids': x['input_ids']},
    y={'outputs': train_y},
    validation_split=0.5,
    batch_size=32,
    epochs=1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(all_data_df['cleaned'], all_data_df['cyberbullying_type'],
                                                    test_size = 0.2, random_state = 42)
print(f'Data Split done.')

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=500000)
vectorizer.fit(X_train)
print(f'Vectorizer fitted.')
print('No. of feature_words: ', len(vectorizer.get_feature_names_out()))

In [ ]:
X_train = vectoriser.transform(X_train)
X_test  = vectoriser.transform(X_test)
print(f'Data Transformed.')

In [ ]:
def model_Evaluate(model):
    
    # Predict values for Test dataset
    y_pred = model.predict(X_test)

    # Print the evaluation metrics for the dataset.
    print(classification_report(y_test, y_pred))
    
    cm=confusion_matrix(y_pred , y_test)
    plt.figure()
    plot_confusion_matrix(cm,figsize=(12,8), hide_ticks=True,cmap=plt.cm.Reds)
    plt.xticks(range(2), ['Negative',  'Positive'], fontsize=16,color='black')
    plt.yticks(range(2), ['Negative', 'Positive'], fontsize=16)
    plt.show()

## Decision Tree Classifier

In [ ]:
from sklearn.metrics import roc_curve, auc
dtc= DecisionTreeClassifier()
dtc.fit(X_train, y_train)
model_Evaluate(dtc)

In [ ]:
from sklearn.preprocessing import label_binarize

# Annahme: dtc ist Ihr Decision Tree Classifier-Modell
classifier = OneVsOneClassifier(dtc)
classifier.fit(X_train, y_train)

# Binarisieren Sie die multiklassen Labels für die Testdaten
y_test_bin = label_binarize(y_test, classes=classifier.classes_)

# Berechnung der ROC-Kurve für jede Klasse
fpr_micro, tpr_micro, _ = roc_curve(y_test_bin.ravel(), y_score.ravel())
roc_auc_micro = auc(fpr_micro, tpr_micro)

# Plotting
plt.figure(figsize=(7, 7))
plt.plot(fpr_micro, tpr_micro, color='darkred', lw=2, label='Micro-average ROC curve (area = {:0.2f})'.format(roc_auc_micro))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([-0.01, 1.0])
plt.ylim([-0.01, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Decision Tree Classifier (Micro-average)')
plt.legend(loc='lower right', fontsize=13)
plt.show()

## Logistic Regression

In [ ]:
lr=LogisticRegression()
lr.fit(X_train, y_train)
model_Evaluate(lr)

In [ ]:
from sklearn.metrics import roc_curve, auc

# Annahme: lr ist Ihr Logistic Regression-Modell
y_score = lr.predict_proba(X_test)

# Binarisieren Sie die multiklassen Labels für die Testdaten
from sklearn.preprocessing import label_binarize
y_test_bin = label_binarize(y_test, classes=lr.classes_)

# Berechnung der Makro-Durchschnitt-ROC-Kurve
fpr_macro = dict()
tpr_macro = dict()
roc_auc_macro = dict()

for i in range(len(lr.classes_)):
    fpr_macro[i], tpr_macro[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
    roc_auc_macro[i] = auc(fpr_macro[i], tpr_macro[i])

# Plotting
plt.figure(figsize=(7, 7))
for i in range(len(lr.classes_)):
    plt.plot(fpr_macro[i], tpr_macro[i], lw=2, label='ROC curve (area = {:0.2f}) for class {}'.format(roc_auc_macro[i], i))

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([-0.01, 1.0])
plt.ylim([-0.01, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Logistic Regression (Macro-average)')
plt.legend(loc='lower right', fontsize=13)
plt.show()

## KNeighbors Clssifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()

knn.fit(X_train,y_train)

model_Evaluate(knn)

In [ ]:
from sklearn.metrics import roc_curve, auc

# Annahme: knn ist Ihr K-Nearest Neighbors-Modell
y_score = knn.predict_proba(X_test)

# Binarisieren Sie die multiklassen Labels für die Testdaten
from sklearn.preprocessing import label_binarize
import numpy as np

n_classes = len(np.unique(y_test))  # Annahme: Sie verwenden NumPy für y_test
y_test_bin = label_binarize(y_test, classes=np.arange(n_classes))

# Berechnung der Makro-Durchschnitt-ROC-Kurve
fpr_macro = dict()
tpr_macro = dict()
roc_auc_macro = dict()

for i in range(n_classes):
    fpr_macro[i], tpr_macro[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
    roc_auc_macro[i] = auc(fpr_macro[i], tpr_macro[i])

# Plotting
plt.figure(figsize=(7, 7))
for i in range(n_classes):
    plt.plot(fpr_macro[i], tpr_macro[i], lw=2, label='ROC curve (area = {:0.2f}) for class {}'.format(roc_auc_macro[i], i))

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([-0.01, 1.0])
plt.ylim([-0.01, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('K-Nearest Neighbors (Macro-average)')
plt.legend(loc='lower right', fontsize=13)
plt.show()

## AdaBoost Classifier

In [ ]:
abc = AdaBoostClassifier()
abc.fit(X_train,y_train)
model_Evaluate(abc)

In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
import numpy as np

# Annahme: abc ist Ihr AdaBoost Classifier-Modell
y_score = abc.predict_proba(X_test)

# Binarisieren Sie die multiklassen Labels für die Testdaten
n_classes = len(np.unique(y_test))
y_test_bin = label_binarize(y_test, classes=np.arange(n_classes))

# Berechnung der Mikro-Durchschnitt-ROC-Kurve
fpr_micro, tpr_micro, _ = roc_curve(y_test_bin.ravel(), y_score.ravel())
roc_auc_micro = auc(fpr_micro, tpr_micro)

# Berechnung der Makro-Durchschnitt-ROC-Kurve
fpr_macro = dict()
tpr_macro = dict()
roc_auc_macro = dict()

for i in range(n_classes):
    fpr_macro[i], tpr_macro[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
    roc_auc_macro[i] = auc(fpr_macro[i], tpr_macro[i])

# Plotting
plt.figure(figsize=(7, 7))
for i in range(n_classes):
    plt.plot(fpr_macro[i], tpr_macro[i], lw=2, label='ROC curve (area = {:0.2f}) for class {}'.format(roc_auc_macro[i], i))

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([-0.01, 1.0])
plt.ylim([-0.01, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('AdaBoost Classifier (Macro-average)')
plt.legend(loc='lower right', fontsize=13)
plt.show()

## MultiNomial NB

In [ ]:
mnb = MultinomialNB()
mnb.fit(X_train,y_train)
model_Evaluate(mnb)

In [ ]:
from sklearn.metrics import roc_curve, auc

# Annahme: mnb ist Ihr Multi-Nomial Naive Bayes-Modell
y_score = mnb.predict_proba(X_test)

# Binarisieren Sie die multiklassen Labels für die Testdaten
n_classes = len(np.unique(y_test))
y_test_bin = label_binarize(y_test, classes=np.arange(n_classes))

# Berechnung der Mikro-Durchschnitt-ROC-Kurve
fpr_micro, tpr_micro, _ = roc_curve(y_test_bin.ravel(), y_score.ravel())
roc_auc_micro = auc(fpr_micro, tpr_micro)

# Plotting
f, axes = plt.subplots(1, 1, figsize=(7, 7))
axes.plot(fpr_micro, tpr_micro, color='darkred', lw=2, label='Micro-average ROC curve (area = {:0.2f})'.format(roc_auc_micro))
axes.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
axes.set(xlim=[-0.01, 1.0], ylim=[-0.01, 1.05])
axes.set(xlabel='False Positive Rate', ylabel='True Positive Rate', title='Multi-Nomial Naive Bayes (Micro-average)')
axes.legend(loc='lower right', fontsize=13)
plt.show()

## Gradient Boosting Classifier

In [ ]:
gbc = GradientBoostingClassifier()
gbc.fit(X_train,y_train)
model_Evaluate(gbc)

In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
import numpy as np
import matplotlib.pyplot as plt
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import GradientBoostingClassifier

# Annahme: gbc ist Ihr Gradient Boosting Classifier-Modell
model = OneVsRestClassifier(GradientBoostingClassifier())
y_score = model.fit(X_train, y_train).decision_function(X_test)

# Binarisieren Sie die multiklassen Labels für die Testdaten
n_classes = len(np.unique(y_test))
y_test_bin = label_binarize(y_test, classes=np.arange(n_classes))

# Berechnung der ROC-Kurve für jede Klasse
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plotting
plt.figure(figsize=(7, 7))
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], lw=2, label='ROC curve (area = {:0.2f}) for class {}'.format(roc_auc[i], i))

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([-0.01, 1.0])
plt.ylim([-0.01, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Gradient Boosting Classifier (One-vs-Rest ROC)')
plt.legend(loc='lower right', fontsize=13)
plt.show()


## Random Forest Clssifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier as RFC
rfc = RFC(random_state=42)

rfc.fit(X_train,y_train)

model_Evaluate(rfc)

In [ ]:
pred_rfc = rfc.predict_proba(X_test)[:,1]
fpr_rfc,tpr_rfc,_ = roc_curve(y_test.values,pred_rfc)
roc_auc_rfc = auc(fpr_rfc,tpr_rfc)


f, axes = plt.subplots(1, 1,figsize=(7,7))
axes.plot(fpr_rfc, tpr_rfc, color='darkred', lw=2, label='ROC curve (area = {:0.2f})'.format(roc_auc_rfc))
axes.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
axes.set(xlim=[-0.01, 1.0], ylim=[-0.01, 1.05])
axes.set(xlabel ='False Positive Rate', ylabel = 'True Positive Rate', title = 'Random Forest Classifer')
axes.legend(loc='lower right', fontsize=13)

## SVC

In [ ]:
from sklearn.svm import SVC
svm=SVC()
svm.fit(X_train,y_train)
model_Evaluate(svm)